# Correlation Analysis

## Load Data

In [3]:
from typing import Any

import polars as pl
import pandas as pd
from src.energy_forecast.config import RAW_DATA_DIR

df_weather = pl.read_csv(RAW_DATA_DIR / f"weather_daily.csv").with_columns(
    pl.col("time").str.to_datetime().alias("datetime"))
df_holidays = pl.read_csv(RAW_DATA_DIR / "holidays.csv").with_columns(pl.col("start").str.to_date(),
                                                                      pl.col("end").str.to_date(strict=False))
df_cities = pl.read_csv(RAW_DATA_DIR / "cities.csv")

# META DATA
df_meta_l = pl.read_csv(RAW_DATA_DIR / "legacy_meta.csv").with_columns(pl.col("plz").str.strip_chars())
df_meta_dh = pl.read_csv(RAW_DATA_DIR / "dh_meta.csv").rename({"eco_u_id": "id"})
df_meta_k = pl.read_csv(RAW_DATA_DIR / "kinergy_meta.csv")

df_meta = pl.concat([df_meta_l.cast({"plz": pl.Int64}).rename(
    {"qmbehfl": "heated_area", "anzlwhg": "anzahlwhg", "adresse": "address"}).with_columns(
    pl.lit("gas").alias("primary_energy")),
    df_meta_dh.rename({"postal_code": "plz", "city": "ort"}),
    df_meta_k.rename({"name": "address"})],
    how="diagonal")

holiday_dict = {"BE": [], "HH": [], "MV": [], "BY": [], "SH": []}
for row in df_holidays.iter_rows():
    if row[1] is not None and row[2] is not None:
        span = pd.date_range(row[1], row[2], freq="D")
        holiday_dict[row[0]].extend(span)
    elif row[1] is not None:
        holiday_dict[row[0]].extend([row[1]])

In [78]:
from src.energy_forecast.config import PROCESSED_DATA_DIR

df_daily = pl.read_csv(PROCESSED_DATA_DIR / "dataset_daily.csv").with_columns(pl.col("datetime").str.to_datetime())
df_hourly = pl.read_csv(PROCESSED_DATA_DIR / "dataset_hourly.csv")

In [79]:
attributes = ["diff", 'hum_avg', 'hum_min', 'hum_max', 'tavg', 'tmin', 'tmax', 'prcp', 'snow', 'wdir', 'wspd', 'wpgt',
              'pres', 'tsun', "holiday"]


def add_holidays(df):
    return df.join(df_cities.select(["plz", "state"]), on="plz", how="left").drop_nulls(["state"]).with_columns(
        pl.struct(["state", "datetime"]).map_elements(lambda x: 1 if x["datetime"] in holiday_dict[x["state"]] else 0,
                                                      return_dtype=pl.Int64).alias("holiday"))


def add_meta(df):
    df = df.join(df_meta, on="id", how="left").join(df_weather, on=["datetime", "plz"], how="left")
    return add_holidays(df)


attributes_ha = attributes + ["heated_area", "anzahlwhg"]
df_daily = add_meta(df_daily).select(["id", "datetime", "primary_energy"] + attributes_ha)

In [80]:
df_daily.write_csv(PROCESSED_DATA_DIR / "dataset_daily_feat.csv")

In [81]:
df_daily = pl.read_csv(PROCESSED_DATA_DIR / "dataset_daily_feat.csv").cast(
    {"heated_area": pl.Float64, "anzahlwhg": pl.Int64})
df_daily

id,datetime,primary_energy,diff,hum_avg,hum_min,hum_max,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun,holiday,heated_area,anzahlwhg
str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,i64
"""573a7d1e-de3f-49e1-828b-07d463…","""2022-10-01T00:00:00.000000""","""district heating""",295.0,85.666667,73.0,96.0,11.8,9.6,15.9,18.3,0.0,201.0,18.7,58.0,1003.3,156.0,1,null,null
"""573a7d1e-de3f-49e1-828b-07d463…","""2022-10-02T00:00:00.000000""","""district heating""",282.0,86.333333,66.0,96.0,12.4,9.3,16.4,11.4,0.0,264.0,15.1,45.7,1013.8,318.0,1,null,null
"""573a7d1e-de3f-49e1-828b-07d463…","""2022-10-03T00:00:00.000000""","""district heating""",270.0,88.333333,74.0,97.0,12.7,8.1,16.2,0.1,0.0,279.0,13.7,45.7,1023.0,138.0,1,null,null
"""573a7d1e-de3f-49e1-828b-07d463…","""2022-10-04T00:00:00.000000""","""district heating""",229.0,83.0,59.0,100.0,12.9,8.8,17.8,0.0,0.0,223.0,11.5,35.3,1019.8,354.0,1,null,null
"""573a7d1e-de3f-49e1-828b-07d463…","""2022-10-05T00:00:00.000000""","""district heating""",223.0,74.833333,68.0,81.0,15.3,12.3,17.6,0.0,0.0,213.0,22.7,58.0,1013.9,0.0,1,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""400965GVG""","""2022-03-26T00:00:00.000000""","""gas""",364.3,83.916667,71.0,99.0,7.8,2.2,12.7,0.0,0.0,280.0,13.3,42.1,1030.5,318.0,1,6139.0,0
"""400965GVG""","""2022-03-27T00:00:00.000000""","""gas""",365.5,86.041667,55.0,100.0,6.6,1.0,13.5,0.0,0.0,281.0,10.1,26.6,1031.0,504.0,1,6139.0,0
"""400965GVG""","""2022-03-28T00:00:00.000000""","""gas""",401.8,84.291667,71.0,95.0,6.4,4.4,9.5,0.0,0.0,297.0,16.9,40.7,1022.1,24.0,1,6139.0,0


From the [meteostat](https://dev.meteostat.net/python/daily.html#api) documentation:

Column	Description	Type

station	The Meteostat ID of the weather station (only if query refers to multiple stations)	String

time	The date	Datetime64

tavg	The average air temperature in °C	Float64

tmin	The minimum air temperature in °C	Float64

tmax	The maximum air temperature in °C	Float64

prcp	The daily precipitation total in mm	Float64

snow	The snow depth in mm	Float64

wdir	The average wind direction in degrees (°)	Float64

wspd	The average wind speed in km/h	Float64

wpgt	The peak wind gust in km/h	Float64

pres	The average sea-level air pressure in hPa	Float64

tsun	The daily sunshine total in minutes (m)	Float64

## Multiple Linear Regression with OLS

### All data (gas + district heating)

In [118]:
import statsmodels.api as sm


def get_p_vals(df: pl.DataFrame, list_cols: list):
    attr_list = list(set(list_cols) - {"diff"})
    X = df.select(attr_list).to_numpy()
    y = df.select(pl.col("diff")).to_numpy()
    X2 = sm.add_constant(X)
    est = sm.OLS(y, X2)
    est2 = est.fit()
    return est2, est2.rsquared, est2.rsquared_adj, est2.summary2().tables[1]["P>|t|"].tolist()


significance_level = 0.05
attributes = ["diff", 'hum_avg', 'hum_min', 'hum_max', 'tavg', 'tmin', 'tmax', 'prcp', 'snow', 'wdir', 'wspd', 'wpgt',
              'pres', 'tsun', "holiday"]


def get_significant_features(df, attributes):
    est, rsquared, rsquared_adj, p_vals = get_p_vals(df, attributes)
    print("r²: ", rsquared)
    print("r² adjusted: ", rsquared_adj)
    return pl.DataFrame({"attributes": attributes, "p-value": p_vals}).sort("p-value").filter(
        pl.col("p-value") < significance_level)


def iterative_feature_removal(df, attributes):
    print(f"Starting with {len(attributes)} features: {attributes}")
    for i in range(5):
        df_significant_feat = get_significant_features(df, attributes)
        print(f"{len(df_significant_feat)} significant features: ")
        for row in df_significant_feat.iter_rows():
            print(row)
        print("\n")
        attributes = df_significant_feat["attributes"].to_list()
        if "diff" not in attributes: attributes.append("diff")


iterative_feature_removal(df_daily.drop_nulls(["snow", "tsun", "wpgt"]), attributes)

Starting with 15 features: ['diff', 'hum_avg', 'hum_min', 'hum_max', 'tavg', 'tmin', 'tmax', 'prcp', 'snow', 'wdir', 'wspd', 'wpgt', 'pres', 'tsun', 'holiday']
r²:  0.08293410027166759
r² adjusted:  0.08278081617567556
12 significant features: 
('tsun', 2.3248772219219265e-42)
('wpgt', 6.448966830396469e-22)
('tmin', 1.576794600522922e-20)
('tavg', 3.3925899105532997e-18)
('diff', 6.76156252067693e-13)
('wspd', 1.4679433816843945e-12)
('tmax', 5.006549850511624e-12)
('hum_avg', 4.1359270652096037e-07)
('wdir', 9.273051716869181e-06)
('pres', 2.6055237878595762e-05)
('snow', 0.000862073953814618)
('holiday', 0.009362907939194774)


r²:  0.08240497822449633
r² adjusted:  0.08228447554739293
10 significant features: 
('snow', 3.0457644971059025e-37)
('tmin', 8.299531501687457e-25)
('tmax', 1.5785053699081775e-19)
('pres', 1.1362697642799555e-16)
('holiday', 1.4291338988333287e-16)
('tsun', 2.4113346229000358e-14)
('hum_avg', 8.28735836455988e-13)
('wpgt', 1.043276143394532e-07)
('wspd', 0

### All data (gas + district heating, only data with known heated area)

In [56]:
attributes_ha

['diff',
 'hum_avg',
 'hum_min',
 'hum_max',
 'tavg',
 'tmin',
 'tmax',
 'prcp',
 'snow',
 'wdir',
 'wspd',
 'wpgt',
 'pres',
 'tsun',
 'holiday',
 'heated_area',
 'anzahlwhg']

In [119]:
iterative_feature_removal(df_daily.drop_nulls(["snow", "tsun", "wpgt", "heated_area", "anzahlwhg"]), attributes_ha)

Starting with 17 features: ['diff', 'hum_avg', 'hum_min', 'hum_max', 'tavg', 'tmin', 'tmax', 'prcp', 'snow', 'wdir', 'wspd', 'wpgt', 'pres', 'tsun', 'holiday', 'heated_area', 'anzahlwhg']
r²:  0.3225055080038579
r² adjusted:  0.3223318610375473
12 significant features: 
('hum_min', 0.0)
('hum_max', 0.0)
('wpgt', 2.1515836795528738e-52)
('holiday', 8.872520288628385e-15)
('snow', 4.966359737166641e-12)
('tsun', 2.054629326447791e-08)
('anzahlwhg', 3.592231827208844e-08)
('tavg', 6.027524783014341e-08)
('tmax', 1.3512985062069032e-07)
('tmin', 3.861297896379728e-05)
('prcp', 0.0007159591024491775)
('heated_area', 0.01819116385837252)


r²:  0.32134039462140507
r² adjusted:  0.3212099437850222
9 significant features: 
('wpgt', 0.0)
('holiday', 0.0)
('heated_area', 3.7149550669507395e-48)
('hum_min', 1.6824928335549718e-28)
('anzahlwhg', 1.0570671995014408e-12)
('tmax', 1.8603810092709733e-06)
('tsun', 0.00018146441011508707)
('tavg', 0.0014577206593262267)
('prcp', 0.021146717665778546)



### Gas Data

In [120]:
attributes = ["diff", 'hum_avg', 'hum_min', 'hum_max', 'tavg', 'tmin', 'tmax', 'prcp', 'snow', 'wdir', 'wspd', 'wpgt',
              'pres', 'tsun', "holiday"]

iterative_feature_removal(df_daily.filter(pl.col("primary_energy") == "gas").drop_nulls(["snow", "tsun", "wpgt"]),
                         attributes)

Starting with 15 features: ['diff', 'hum_avg', 'hum_min', 'hum_max', 'tavg', 'tmin', 'tmax', 'prcp', 'snow', 'wdir', 'wspd', 'wpgt', 'pres', 'tsun', 'holiday']
r²:  0.057509056015120574
r² adjusted:  0.057292821059682475
10 significant features: 
('tsun', 5.953088618250138e-107)
('tmax', 1.7642489492759807e-06)
('tmin', 2.572075185217899e-06)
('tavg', 8.936302981787766e-06)
('snow', 4.03170381693163e-05)
('wdir', 0.00019005102307918182)
('wpgt', 0.000830880713482508)
('holiday', 0.004946016040061716)
('wspd', 0.013905368148106428)
('pres', 0.034963978277115206)


r²:  0.057112191406720525
r² adjusted:  0.05695768295795478
7 significant features: 
('pres', 3.4701655482279396e-102)
('wspd', 6.035820088289768e-07)
('tavg', 6.760154954624523e-07)
('tmin', 0.0001073157062462121)
('wpgt', 0.0012163006054174974)
('wdir', 0.012411988132782386)
('snow', 0.014704776305916292)


r²:  0.049821204379093764
r² adjusted:  0.04971221749488741
6 significant features: 
('tmin', 1.3679729047632398e-118)


### Gas Data (with heated area available)

In [121]:
iterative_feature_removal(df_daily.filter(pl.col("primary_energy") == "gas").drop_nulls(["snow", "tsun", "wpgt", "heated_area", "anzahlwhg"]),
    attributes_ha)

Starting with 17 features: ['diff', 'hum_avg', 'hum_min', 'hum_max', 'tavg', 'tmin', 'tmax', 'prcp', 'snow', 'wdir', 'wspd', 'wpgt', 'pres', 'tsun', 'holiday', 'heated_area', 'anzahlwhg']
r²:  0.3206415334462156
r² adjusted:  0.32046339654680944
11 significant features: 
('hum_min', 0.0)
('hum_max', 0.0)
('wpgt', 8.711196608257125e-60)
('holiday', 1.7120508524856693e-12)
('snow', 2.827550153017439e-10)
('anzahlwhg', 5.89757720233844e-09)
('tavg', 2.7513625521606226e-07)
('tsun', 6.02139048655486e-07)
('tmax', 9.053228030780217e-06)
('tmin', 0.001654209881566006)
('prcp', 0.022690583279264757)


r²:  0.19484546534405778
r² adjusted:  0.1947003306449031
6 significant features: 
('wpgt', 0.0)
('prcp', 4.622714980475514e-92)
('hum_min', 7.936742912583737e-13)
('anzahlwhg', 3.459871490252611e-09)
('holiday', 1.2332465262919764e-05)
('tavg', 0.0010743011135946664)


r²:  0.1946367329904436
r² adjusted:  0.1945575545735917
6 significant features: 
('hum_min', 0.0)
('holiday', 0.0)
('tavg', 1.

### District Heating Data

In [123]:
attributes = ["diff", 'hum_avg', 'hum_min', 'hum_max', 'tavg', 'tmin', 'tmax', 'prcp', 'snow', 'wdir', 'wspd', 'wpgt',
              'pres', 'tsun', "holiday"]
iterative_feature_removal(df_daily.filter(pl.col("primary_energy") == "district heating").drop_nulls(["snow", "tsun", "wpgt"]),
    attributes)

Starting with 15 features: ['diff', 'hum_avg', 'hum_min', 'hum_max', 'tavg', 'tmin', 'tmax', 'prcp', 'snow', 'wdir', 'wspd', 'wpgt', 'pres', 'tsun', 'holiday']
r²:  0.1832768503224298
r² adjusted:  0.18277365426136893
10 significant features: 
('snow', 1.835186651717902e-09)
('wspd', 1.1321343630267453e-07)
('wpgt', 3.324532611704073e-06)
('diff', 4.7636583516860146e-06)
('prcp', 6.948679135769155e-05)
('wdir', 0.000555664494320061)
('tmax', 0.0010160027822977845)
('tsun', 0.02640504269360093)
('hum_avg', 0.0285478778132571)
('tavg', 0.03287218175354488)


r²:  0.18236901097566893
r² adjusted:  0.18204523946470375
8 significant features: 
('snow', 1.171040810840072e-64)
('prcp', 4.753176794751752e-20)
('tmax', 5.11064775026878e-08)
('tsun', 2.0432447038646655e-07)
('diff', 3.267026692549305e-06)
('wdir', 0.0016175799747712783)
('wpgt', 0.012687706903415157)
('tavg', 0.013625788958580767)


r²:  0.18080231671857938
r² adjusted:  0.1805500341060422
7 significant features: 
('snow', 0.0)


### District Heating Data (with heated area available)

In [124]:
iterative_feature_removal(df_daily.filter(pl.col("primary_energy") == "district heating").drop_nulls(["snow", "tsun", "wpgt", "heated_area", "anzahlwhg"]),
    attributes_ha)

Starting with 17 features: ['diff', 'hum_avg', 'hum_min', 'hum_max', 'tavg', 'tmin', 'tmax', 'prcp', 'snow', 'wdir', 'wspd', 'wpgt', 'pres', 'tsun', 'holiday', 'heated_area', 'anzahlwhg']
r²:  0.6710225528701727
r² adjusted:  0.6672330358405995
9 significant features: 
('hum_min', 1.0081084569921094e-102)
('snow', 7.630434427116196e-06)
('tavg', 0.00014979344900251614)
('holiday', 0.00028837204604805215)
('anzahlwhg', 0.0025236512948980934)
('tmin', 0.004391003897723508)
('heated_area', 0.004468921446093297)
('tmax', 0.007166949179288967)
('wspd', 0.02181438396365982)


r²:  0.6646107174642693
r² adjusted:  0.6624484656427638
8 significant features: 
('snow', 3.1071547296443247e-107)
('holiday', 2.606708079031251e-06)
('tmax', 0.00016805577769396138)
('heated_area', 0.00024906467422165313)
('hum_min', 0.002202538229642393)
('tavg', 0.023150518773640445)
('anzahlwhg', 0.02716785493681318)
('diff', 0.04392433070910674)


r²:  0.6587655950485284
r² adjusted:  0.6570569821481991
6 signific